In [33]:
import pandas as pd
import numpy as np
import math
from collections import Counter

#Reading data

df_training = pd.read_csv("Training_Data.csv")
df_leader = pd.read_csv("Test_Data.csv")

print("data reading complete")

#df_training = df_training.drop([' '],axis=1)
df_training.drop(df_training.columns[[0,1]],axis=1,inplace=True)
df_training_timestamp = df_training.loc[:, 'timestamp']
df_leader_timestamp = df_leader.loc[:, 'timestamp']
series_id = df_leader.loc[:, 'series_id']
df_training.drop(df_training.columns[[0]], axis=1, inplace=True)

df_leader.drop(df_leader.columns[[0,1,2]],axis=1,inplace=True)

df_training = df_training.apply(np.log, axis=1)
df_leader = df_leader.apply(np.log, axis=1)

#df_training = np.log(df_training).dropna()

df_training = pd.concat((df_training_timestamp,df_training),axis =1)
df_leader = pd.concat((df_leader_timestamp,df_leader),axis =1)

print(df_training)
print(df_leader)



data reading complete
                  timestamp  consumption  temperature
0       2014-12-24 00:00:00    11.531180          NaN
1       2014-12-24 01:00:00    11.570043          NaN
2       2014-12-24 02:00:00    11.425395          NaN
3       2014-12-24 03:00:00    11.456077          NaN
4       2014-12-24 04:00:00    11.482227          NaN
5       2014-12-24 05:00:00    11.600520          NaN
6       2014-12-24 06:00:00    11.428562          NaN
7       2014-12-24 07:00:00    11.521839          NaN
8       2014-12-24 08:00:00    11.511144          NaN
9       2014-12-24 09:00:00    11.420986          NaN
10      2014-12-24 10:00:00    11.437447          NaN
11      2014-12-24 11:00:00    11.360399          NaN
12      2014-12-24 12:00:00    11.564806          NaN
13      2014-12-24 13:00:00    11.347652          NaN
14      2014-12-24 14:00:00    11.408107          NaN
15      2014-12-24 15:00:00    11.476701          NaN
16      2014-12-24 16:00:00    11.322000          NaN
17    

In [ ]:
#replace all missing values with mean of existing values in training set
#same can be done for testing set but data leakage can occur

df_training = df_training.fillna(df_training.mean())

df_leader = df_leader.fillna(df_leader.mean())

print("data processed")

#df_training = (df_training - df_training.mean()) / (df_training.max() - df_training.min())
#for head in df_training:
    #df_training[head] = np.sqrt(df_training[head])

#print("data normalised")
print(df_training)

In [21]:
#general informations

n_people = df_training.shape[0]
n_features = df_training.shape[1]-1
print("Total number of people: {}".format(n_people))
print("Number of features: {}".format(n_features))

Total number of people: 80000
Number of features: 46


In [22]:
#preparing data

# Extract feature (X) and target (y) columns
feature_cols = list(df_training.columns[:-1])  # all columns but last are features
target_col = df_training.columns[-1] # last column is the target/label
print("Feature column(s):-\n{}".format(feature_cols))
print( "Target column: {}".format(target_col))

feature_cols_leader = list(df_leader.columns[:])  # all columns but last are features

X_all = df_training[feature_cols]  # feature values for all people
y_all = df_training[target_col]  # corresponding targets/labels

X_leader = df_leader[feature_cols_leader]

print ("\nFeature values:-")
print(X_all.head())  # print the first 5 rows

X_all = X_all.convert_objects(convert_numeric=True)
X_leader = X_leader.convert_objects(convert_numeric=True)


for head in X_all:
    series = X_all[head]
    mean = series.mean()
    maximum = series.max()
    minimum = series.min()
    series = (series - minimum)/(maximum - minimum)
    X_all[head] = series
    
for head in X_leader:
    series = X_leader[head]
    mean = series.mean()
    maximum = series.max()
    minimum = series.min()
    series = (series - minimum)/(maximum - minimum)
    X_leader[head] = series
    

print("data normalised")


Feature column(s):-
['mvar1', 'mvar2', 'mvar3', 'mvar4', 'mvar5', 'mvar6', 'mvar7', 'mvar8', 'mvar9', 'mvar10', 'mvar11', 'mvar12', 'mvar13', 'mvar14', 'mvar15', 'mvar16', 'mvar17', 'mvar18', 'mvar19', 'mvar20', 'mvar21', 'mvar22', 'mvar23', 'mvar24', 'mvar25', 'mvar26', 'mvar27', 'mvar28', 'mvar29', 'mvar30', 'mvar31', 'mvar32', 'mvar33', 'mvar34', 'mvar35', 'mvar36', 'mvar37', 'mvar38', 'mvar39', 'mvar40', 'mvar41', 'mvar42', 'mvar43', 'mvar44', 'mvar45', 'mvar46']
Target column: default_ind

Feature values:-
  mvar1   mvar2  mvar3  mvar4  mvar5 mvar6  mvar7 mvar8  mvar9 mvar10  ...    \
0  1696  1.6541  0.000    0.0    0.0     0   6015   322  40369  18414  ...     
1  1846  0.8095  0.000    0.0    0.0   102   7532  3171  18234  13664  ...     
2  1745  0.4001  0.000    0.0    0.0     0   2536     0      0   2536  ...     
3  1739  0.2193  0.000    0.0    0.0  1982  26440  4955  20316  37013  ...     
4  1787  0.0118  0.225    0.0    0.0  5451   5494  5494   7987   4696  ...     

  

C:\Users\Utkarsh\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
C:\Users\Utkarsh\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


data normalised


In [23]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['C', 'L'], [1, 0])

        # If still non-numeric, convert to one or more dummy variables
        #if col_data.dtype == object:
            #col_data = pd.get_dummies(col_data, prefix=col)

        outX = outX.join(col_data)  # collect columns in output dataframe

    return outX

X_all = preprocess_features(X_all)
X_leader = preprocess_features(X_leader)
mvar47 = mvar47.replace('C',0)
mvar47 = mvar47.replace('L',1)
mvar47leader = mvar47leader.replace('C',0)
mvar47leader = mvar47leader.replace('L',1)

X_all = pd.concat((X_all,mvar47),axis =1)
X_leader = pd.concat((X_leader,mvar47leader),axis =1)

print("Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns)))

Processed feature columns (47):-
['mvar1', 'mvar2', 'mvar3', 'mvar4', 'mvar5', 'mvar6', 'mvar7', 'mvar8', 'mvar9', 'mvar10', 'mvar11', 'mvar12', 'mvar13', 'mvar14', 'mvar15', 'mvar16', 'mvar17', 'mvar18', 'mvar19', 'mvar20', 'mvar21', 'mvar22', 'mvar23', 'mvar24', 'mvar25', 'mvar26', 'mvar27', 'mvar28', 'mvar29', 'mvar30', 'mvar31', 'mvar32', 'mvar33', 'mvar34', 'mvar35', 'mvar36', 'mvar37', 'mvar38', 'mvar39', 'mvar40', 'mvar41', 'mvar42', 'mvar43', 'mvar44', 'mvar45', 'mvar46', 'mvar47']


In [24]:
# preparing number of training and test samples
num_all = df_training.shape[0]  # same as len(df_training)
num_train = 72000 # about 90% of the data
num_test = num_all - num_train

# Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset
from sklearn import cross_validation

X_train, X_test, y_train, y_test = cross_validation.train_test_split(X_all, y_all, test_size=num_test)
print("Training set: {} samples".format(X_train.shape[0]))
print ("Test set: {} samples".format(X_test.shape[0]))
# Note: If you need a validation set, extract it from within training data

Training set: 72000 samples
Test set: 8000 samples


In [25]:
# Train a model
import time

def train_classifier(clf, X_train, y_train):
    print("Training {}...".format(clf.__class__.__name__))
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    print ("Done!\nTraining time (secs): {:.3f}".format(end - start))

from sklearn import ensemble
from sklearn.svm import SVC

clf = ensemble.RandomForestClassifier(criterion='entropy',min_samples_split= 25)
#clf = SVC(gamma='auto')

# Fit model to training data
train_classifier(clf, X_train, y_train) 

Training RandomForestClassifier...
Done!
Training time (secs): 6.098


In [26]:
from sklearn.metrics import f1_score

def predict_labels(clf, features, target):
    print("Predicting labels using {}...".format(clf.__class__.__name__))
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    print( "Done!\nPrediction time (secs): {:.3f}".format(end - start))
    return f1_score(target.values, y_pred, pos_label=1)

train_f1_score = predict_labels(clf, X_train, y_train)
print ("F1 score for training set: {}".format(train_f1_score))

Predicting labels using RandomForestClassifier...
Done!
Prediction time (secs): 0.301
F1 score for training set: 0.7765848013372766


In [27]:
# Predict on test data
print("F1 score for test set: {}".format(predict_labels(clf, X_test, y_test)))

predict_leader = clf.predict(X_leader)

data = {'default_ind':predict_leader}

predict_leader = pd.DataFrame.from_dict(data)

LeaderBoardFinal = pd.concat((application_key,predict_leader),axis =1)

print(LeaderBoardFinal.shape)

LeaderBoardFinal.to_csv('Flubbers_IITRookree_1.csv',index=False,header=False)


#Now just by changing the classifier we can compare the f1 scores

Predicting labels using RandomForestClassifier...
Done!
Prediction time (secs): 0.062
F1 score for test set: 0.5236818588025022
(25000, 2)


In [10]:
# Train and predict using different training set sizes
def train_predict(clf, X_train, y_train, X_test, y_test):
    print("------------------------------------------")
    print("Training set size: {}".format(len(X_train)))
    train_classifier(clf, X_train, y_train)
    print( "F1 score for training set: {}".format(predict_labels(clf, X_train, y_train)))
    print ("F1 score for test set: {}".format(predict_labels(clf, X_test, y_test)))

train_predict(clf, X_train.sample(n=70000, random_state=70000), y_train.sample(n=70000, random_state=70000), X_test, y_test)
train_predict(clf, X_train.sample(n=60000, random_state=60000), y_train.sample(n=60000, random_state=60000), X_test, y_test)
# Note: Keep the test set constant

------------------------------------------
Training set size: 70000
Training RandomForestClassifier...
Done!
Training time (secs): 5.942
Predicting labels using RandomForestClassifier...
Done!
Prediction time (secs): 0.300
F1 score for training set: 0.7722247964519758
Predicting labels using RandomForestClassifier...
Done!
Prediction time (secs): 0.039
F1 score for test set: 0.5013824884792627
------------------------------------------
Training set size: 60000
Training RandomForestClassifier...
Done!
Training time (secs): 5.281
Predicting labels using RandomForestClassifier...
Done!
Prediction time (secs): 0.240
F1 score for training set: 0.7703651957930687
Predicting labels using RandomForestClassifier...
Done!
Prediction time (secs): 0.031
F1 score for test set: 0.5127105666156202


In [15]:
from sklearn import tree

dtc = tree.DecisionTreeClassifier(min_samples_split=10)
train_predict(dtc, X_train, y_train, X_test, y_test)
train_predict(dtc, X_train.sample(n=200, random_state=202), y_train.sample(n=200, random_state=202), X_test, y_test)
train_predict(dtc, X_train.sample(n=100, random_state=102), y_train.sample(n=100, random_state=102), X_test, y_test)

------------------------------------------
Training set size: 72000
Training DecisionTreeClassifier...
Done!
Training time (secs): 5.406
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.074
F1 score for training set: 0.9038870069472354
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.008
F1 score for test set: 0.45445292620865146
------------------------------------------
Training set size: 200
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.005
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.002
F1 score for training set: 0.920353982300885
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.007
F1 score for test set: 0.39015817223198596
------------------------------------------
Training set size: 100
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.000
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time 

In [ ]:
#choosing the best model

from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer

parameters = {'max_depth': (1, 2, 3, 4, 5, 6, 7, 8, 9, 10), 'n_estimators': (100, 125, 150, 500)}
sss = cross_validation.StratifiedShuffleSplit(y_train, test_size=num_test)
gs = GridSearchCV(estimator=clf, n_jobs=-1, scoring=make_scorer(f1_score, pos_label=1), param_grid=parameters,
                  cv=sss)
gs.fit(X_train, y_train)
best_estimator = gs.best_estimator_
print("best estimator:\n{}".format(best_estimator))
print ('')
print("best parameter:\n{}".format(gs.best_params_))
print('')
print("F1 score:\n{}".format(f1_score(y_test, best_estimator.predict(X_test), pos_label=1)))


C:\Users\Utkarsh\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
